In [1]:
import sys
sys.path.append("/s/ls4/users/grartem/RL_robots/CommandClassifier")

In [2]:
import json
from collections import defaultdict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from simpletransformers.ner import NERModel, NERArgs

объединить классы объектов и релейшенов в два класса, вместо пяти, а может и не надо. Если допускать порядок слов

предсказывать несколько возможных вариантов классификаций для токенов. И сохранять активности.

правило прикрутить

и класс экшена уточнять по атрибутам

In [6]:
TEST_FOLD = 0
train_df = {
    "sentence_id": [],
    "words": [],
    "labels": []
}
valid_df = {
    "sentence_id": [],
    "words": [],
    "labels": []
}
test_df = {
    "sentence_id": [],
    "words": [],
    "labels": []
}

label_to_index = ["A", "D", "M", "DH", "O1", "N", "R1", "O2", "R2", "O3", "S", "G"]
train_sentence_counter, valid_sentence_counter = 0, 0
with open("../../Data/Interim/onecmd_v6_20122022_low.jsonl", "r") as f:
    for line in f:
        docData = json.loads(line)
        if docData["subset"]=="train":
            for w_i, word in enumerate(docData["parse"]):
                train_df["sentence_id"].append(train_sentence_counter)
                train_df["words"].append(word["form"])
                train_df["labels"].append(word["attribute_name"][0])
                    
            train_sentence_counter += 1
        elif docData["subset"]=="valid":
            for w_i, word in enumerate(docData["parse"]):
                valid_df["sentence_id"].append(valid_sentence_counter)
                valid_df["words"].append(word["form"])
                valid_df["labels"].append(word["attribute_name"][0])
            valid_sentence_counter += 1
        elif docData["subset"]=="test":
            if docData["fold"]==TEST_FOLD:
                for w_i, word in enumerate(docData["parse"]):
                    test_df["sentence_id"].append(valid_sentence_counter)
                    test_df["words"].append(word["form"])
                    test_df["labels"].append(word["attribute_name"][0])
            else:
                for w_i, word in enumerate(docData["parse"]):
                    train_df["sentence_id"].append(valid_sentence_counter)
                    train_df["words"].append(word["form"])
                    train_df["labels"].append(word["attribute_name"][0])
            valid_sentence_counter += 1
        else:
            print(docData["subset"])
            assert False
train_df = pd.DataFrame(train_df)
valid_df = pd.DataFrame(valid_df)
test_df = pd.DataFrame(test_df)

In [4]:
train_df["labels"].unique()

array(['A', 'O1', 'O', 'R1', 'O2', 'D', 'M', 'DH', 'S', 'G', 'R2', 'O3',
       'N'], dtype=object)

In [12]:
train_df["labels"].unique()

array(['A', 'O1', 'O', 'R1', 'O2', 'D', 'M', 'S', 'G', 'R2', 'O3', 'N',
       'HS', 'DS', 'DH'], dtype=object)

In [7]:
model_args = NERArgs()
model_args.evaluate_during_training = True
model_args.num_train_epochs = 20
model_args.output_dir = "/s/ls4/users/grartem/RL_robots/CommandClassifier/models/CommandParser_tokenCLS_copy-1_ipynb_data5"

model = NERModel(
    "bert", 
    "/s/ls4/groups/g0126/transformers_models/cointegrated/rubert-tiny2",
    #"/s/ls4/users/grartem/RL_robots/CommandClassifier/models/TokenCLS_sepPredictor_ipynb/checkpoint-20664-epoch-9/",
    args=model_args,
    labels=["O"] + ["A", "D", "M", "DH", "O1", "N", "R1", "O2", "R2", "O3", "S", "G"]
)

Some weights of the model checkpoint at /s/ls4/groups/g0126/transformers_models/cointegrated/rubert-tiny2 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForT

In [6]:
# Train the model
model.train_model(train_df, eval_data=valid_df)

  0%|          | 0/29 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 0 of 20:   0%|          | 0/5835 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

...

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: A seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: R1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: D seems not to be NE ta

  0%|          | 0/27 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1686 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: A seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: R1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: D seems not to be NE ta

(116700,
 defaultdict(list,
             {'global_step': [2000,
               4000,
               5835,
               6000,
               8000,
               10000,
               11670,
               12000,
               14000,
               16000,
               17505,
               18000,
               20000,
               22000,
               23340,
               24000,
               26000,
               28000,
               29175,
               30000,
               32000,
               34000,
               35010,
               36000,
               38000,
               40000,
               40845,
               42000,
               44000,
               46000,
               46680,
               48000,
               50000,
               52000,
               52515,
               54000,
               56000,
               58000,
               58350,
               60000,
               62000,
               64000,
               64185,
               6

# Test
надо посчитать отдельно для лейбла и класса

In [8]:
from RobotCommandClassifier import utils
from RobotCommandClassifier import *

In [9]:
model = NERModel(
    "bert", 
    "/s/ls4/users/grartem/RL_robots/CommandClassifier/models/CommandParser_tokenCLS_copy-1_ipynb_data5/checkpoint-116700-epoch-20/",
    #"/s/ls4/users/grartem/RL_robots/CommandClassifier/models/TokenCLS_sepPredictor_ipynb/checkpoint-20664-epoch-9/",
    args=model_args,
    labels=["O"] + ["A", "D", "M", "DH", "O1", "N", "R1", "O2", "R2", "O3", "S", "G"]
)

In [10]:
test_df = pd.read_csv("../../Data/Interim/one_cmd_v4_05072022_low_split.csv")
test_df = test_df[test_df["subset"]=="test"]
predictions, raw_outputs = model.predict(test_df["x"].values)

  0%|          | 0/3 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/165 [00:00<?, ?it/s]

In [9]:
#test_df.loc[:,["action", 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3', 'self', 'gaze']] = (test_df.loc[:,["action", 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3', 'self', 'gaze']] > 0).astype(np.int32)
valid_array = test_df.loc[:,['action', 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3', 'self', 'gaze']]
#valid_array = test_df.loc[:,["action", 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3', 'self', 'gaze']] = (test_df.loc[:,["action", 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3', 'self', 'gaze']] > 0).astype(np.int32)

In [16]:
# Если оцениваю просто наличие атрибутов (бинарная классификация)
pred_array = []
for i in range(len(test_df)):
    y = np.zeros(len(label_to_index))
    for word in predictions[i]:
        word_label = list(word.values())[0]
        if word_label=="O":
            continue
        #y[label_to_index.index(word_label)] = 1
    pred_array.append(y)
pred_array = np.array(pred_array)

In [24]:
# Если оцениваю присваивая эталонную классификацию
pred_array = []
for i in range(len(test_df)):
    y = np.zeros(len(label_to_index))
    actionDetected = False
    for word in predictions[i]:
        word_label = list(word.values())[0]
        if word_label=="O":
            continue
        if word_label=="A":
            actionDetected = True
        #y[label_to_index.index(word_label)] = 1
        if valid_array.iloc[i, label_to_index.index(word_label)] == 0 and word_label!="A":
            y[label_to_index.index(word_label)] = 20
        else:
            y[label_to_index.index(word_label)] = valid_array.iloc[i, label_to_index.index(word_label)]
    if not actionDetected:
        y[0] = 20
    pred_array.append(y)
pred_array = np.array(pred_array)

In [25]:
for i, pred in enumerate(predictions):
    if "A" not in [ list(word.values())[0] for word in pred]:
        print(predictions[i])

[{'лево': 'D'}, {'руля': 'D'}]
[{'направо': 'D'}, {'ой': 'G'}, {'то': 'G'}, {'есть': 'G'}, {'налево': 'D'}]
[{'левее': 'R2'}, {'давай': 'O'}]
[{'правее': 'R2'}, {'давай': 'O2'}]
[{'вперед': 'D'}, {'и': 'D'}, {'с': 'R1'}, {'песней': 'O2'}]
[{'по': 'D'}, {'правую': 'D'}, {'руку': 'D'}]
[{'правый': 'D'}, {'борт': 'D'}]
[{'на': 'D'}, {'запад': 'D'}]
[{'левый': 'D'}, {'борт': 'D'}]
[{'на': 'D'}, {'восток': 'D'}]
[{'по': 'D'}, {'левую': 'D'}, {'руку': 'D'}]
[{'вспышка': 'O1'}, {'справа': 'S'}]
[{'недалеко': 'R1'}, {'от': 'R1'}, {'леса': 'O2'}, {'дерево': 'O1'}, {'сходи': 'O'}, {'к': 'O'}, {'нему': 'O'}]
[{'около': 'R1'}, {'леса': 'O2'}, {'есть': 'O'}, {'дерево': 'O1'}, {'направляйся': 'O'}, {'к': 'O'}, {'этому': 'G'}, {'дереву': 'O1'}]
[{'недалеко': 'R1'}, {'от': 'R1'}, {'ямы': 'O2'}, {'человек': 'O1'}, {'сходи': 'O'}, {'к': 'O'}, {'нему': 'O'}]
[{'около': 'R1'}, {'ямы': 'O2'}, {'человек': 'O1'}, {'будь': 'O'}, {'у': 'O'}, {'него': 'O'}]
[{'возле': 'R1'}, {'коттеджа': 'O2'}, {'растёт': 'O'},

In [26]:
result = utils.calculate_metrics_2(valid_array, pred_array, display=True)

#result_avg = utils.calculate_metrics(test_y_df.iloc[:,1:], predictions_2[:,1:], config={
result_avg = utils.calculate_metrics(valid_array, pred_array, config={
    "report_metrics": ["correct_samples_perc", "avg_macro_f1", "avg_acc", "class_acc", "class_macro_f1"]
})
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

action
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        33
         1.0       1.00      1.00      1.00        18
         2.0       1.00      0.98      0.99        56
         3.0       1.00      0.67      0.80        33
         6.0       1.00      0.98      0.99      1178
        20.0       0.00      0.00      0.00         0

    accuracy                           0.97      1318
   macro avg       0.83      0.77      0.80      1318
weighted avg       1.00      0.97      0.98      1318

direction
              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96      1229
         1.0       1.00      0.06      0.11        33
         2.0       1.00      0.22      0.36        23
         3.0       1.00      0.67      0.80         9
         4.0       1.00      0.88      0.93        24
        20.0       0.00      0.00      0.00         0

    accuracy                           0.92      1318
   mac

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

69.0 	 correct_samples_perc
97.0 	 [action]_acc
80.0 	 [action]_macrof1
92.0 	 [direction]_acc
53.0 	 [direction]_macrof1
100.0 	 [meters]_acc
50.0 	 [meters]_macrof1
99.0 	 [degshours]_acc
50.0 	 [degshours]_macrof1
97.0 	 [object1]_acc
85.0 	 [object1]_macrof1
96.0 	 [nearest]_acc
49.0 	 [nearest]_macrof1
97.0 	 [relation1]_acc
72.0 	 [relation1]_macrof1
97.0 	 [object2]_acc
85.0 	 [object2]_macrof1
94.0 	 [relation2]_acc
71.0 	 [relation2]_macrof1
94.0 	 [object3]_acc
83.0 	 [object3]_macrof1
96.0 	 [self]_acc
49.0 	 [self]_macrof1
93.0 	 [gaze]_acc
48.0 	 [gaze]_macrof1
65.0 	 avg_macro_f1
96.0 	 avg_acc


/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [51]:
with open("/s/ls4/users/grartem/RL_robots/CommandClassifier/models/CommandParser_tokenCLS_copy-1_ipynb_data5/checkpoint-116700-epoch-20/predict_test_one_cmd_v4_05072022_low_split.jsonlines", "w") as f:
    for i, sample in enumerate(predictions):
        words = [list(x.keys())[0] for x in sample]
        labels = [list(x.values())[0] for x in sample]
        activities = [list(x.values())[0] for x in raw_outputs[i]]
        for w_i in range(len(activities)):
            for subw_i in range(len(activities[w_i])):
                activities[w_i][subw_i] = [float(x) for x in activities[w_i][subw_i]]
        text = test_df.iloc[i]["x"]
        f.write(json.dumps([text, words, labels, activities]) + "\n")